In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation


def compute_exp_W(sigma, w):
    theta = np.linalg.norm(w)
    if theta < 1e-8:
        return np.eye(3), np.eye(3)
    A = np.exp(sigma) * np.sin(theta)
    B = np.exp(sigma) * np.cos(theta)
    C = (np.exp(sigma) - 1) / sigma
    wx = np.array([
        [0, -w[2], w[1]],
        [w[2], 0, -w[0]],
        [-w[1], w[0], 0]
    ])
    exp_wx = np.eye(3) + wx * (np.sin(theta) / theta) + (wx @ wx) * ((1 - np.cos(theta)) / np.square(theta))
    W = C * np.eye(3) + ((A * sigma + theta * (1 - B)) / (np.square(sigma) + np.square(theta))) * (wx / theta) + (C - ((sigma * (B - 1) + A * theta) / (np.square(sigma) + np.square(theta)))) * ((wx / theta) @ (wx / theta))
    return exp_wx, W

def compute_residuals_jacobian(zi, pi, s, R, t):
    sr = s * R[2]
    srpi = sr @ pi
    ri = zi - (srpi + t[2])
    ds = -srpi
    dw = np.cross(sr, pi)
    dp = -sr
    Ji = np.hstack([ds, dw, dp])
    return ri, Ji

def levenberg_marquardt(z, p, s=1.0, R=np.eye(3), t=np.zeros(3), max_iter=1000):
    
    mu = 0.01
    previous_cost = float('inf')
    
    print('---------------------------------------------')
    print('Starting optimization')
    print('---------------------------------------------')
    
    for iteration in range(max_iter):
        r = []
        J = []
        for i in range(len(z)):
            ri, Ji = compute_residuals_jacobian(z[i], p[i], s, R, t)
            r.append(ri)
            J.append(Ji)
        r = np.stack(r)
        J = np.stack(J)
        
        cost = np.square(r).sum()
        cost_change = np.abs(previous_cost - cost) / cost
        print(f'iter: {iteration}, cost: {np.square(r).sum()}, cost change: {cost_change}')
        
        if cost_change < 1e-8:
            break
        
        delta = -(np.linalg.inv(J.T @ J + mu * np.eye(7)) @ J.T @ r)
        
        exp_wx, W = compute_exp_W(delta[0], delta[1:4])
        
        t = s * R @ W @ delta[4:7] + t
        s = s * np.exp(delta[0])
        R = R @ exp_wx
        
        if cost < previous_cost:
            mu *= 0.9
        else:
            mu *= 1.1
        previous_cost = cost
    
    return s, R, t

In [ ]:
p_gt = np.random.rand(40, 3)
z_gt = p_gt[:, 2].copy() + np.random.randn(p_gt.shape[0]) / 20

s_es = 0.78
R_es, _ = compute_exp_W(1.0, np.random.rand(3)*2)
p_es_bias = np.random.rand(3, 1)
p_es = (s_es * R_es @ p_gt.T + p_es_bias).T

cmap = matplotlib.cm.get_cmap('jet')
colors = cmap(np.linspace(0, 1, z_gt.size))

In [ ]:
s, R, t = levenberg_marquardt(z_gt, p_es)

In [ ]:
1/s

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
new_p = s * R @ p_es.T + t.reshape(3, 1)
real_alt = new_p.copy()
real_alt[2] = z_gt
ax.scatter(*real_alt, c=colors)
ax.scatter(*new_p, c=colors * 0.7)